In [8]:
## libraries
sys.path.append("../covid_analysis/")
import importlib
import sys
import os
import numpy as np
import pandas as pd 
import seaborn as sns
import logging

## custom libraries

import covid_utils as cu
importlib.reload(cu)


['D:\\ProgramData\\Anaconda3\\envs\\docker\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\aminm\\AppData\\Roaming\\jupyter\\runtime\\kernel-6bffa452-8d51-4916-ba50-a2b96d071775.json']


<module 'covid_utils' from '../covid_analysis\\covid_utils.py'>

In [2]:
# Add logging feature to capture daily changes
logging.basicConfig(filename="covid_compile.log",
                    format='%(asctime)s %(message)s',
                    filemode='a', level=logging.DEBUG)

# Creating an object
logger = logging.getLogger()

In [9]:
## files and directories
dir_input = '../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/'
file_pattern = '*.csv'

## column names to rename
# ref https://www.geeksforgeeks.org/how-to-rename-columns-in-pandas-dataframe/

column_rename = {'Country/Region':'Country_Region', 'Province/State':'Province_State', 'Last Update':'Last_Update','Latitude':'Lat' , 'Longitude':'Long_'}


In [11]:
# read daily reports

combine_df = cu.read_daily_reports(dir_input, file_pattern, rename_col=column_rename)
logger.info(f'Dim of combine df: {str(combine_df.shape)}')
print(combine_df.head())
print(combine_df.columns)

list_group = ['Country_Region', 'Province_State','Admin2','Last_Update']
list_display = ['Confirmed', 'Deaths']
country_summ = cu.summarize_by_county(combine_df, list_group, list_display)
print(country_summ.head())

Total daily reports: 73
   Active Admin2 Combined_Key  Confirmed  Country_Region  Deaths  FIPS  \
0     NaN    NaN          NaN        1.0  Mainland China     NaN   NaN   
1     NaN    NaN          NaN       14.0  Mainland China     NaN   NaN   
2     NaN    NaN          NaN        6.0  Mainland China     NaN   NaN   
3     NaN    NaN          NaN        1.0  Mainland China     NaN   NaN   
4     NaN    NaN          NaN        NaN  Mainland China     NaN   NaN   

       Last_Update  Lat  Long_ Province_State  Recovered  
0  1/22/2020 17:00  NaN    NaN          Anhui        NaN  
1  1/22/2020 17:00  NaN    NaN        Beijing        NaN  
2  1/22/2020 17:00  NaN    NaN      Chongqing        NaN  
3  1/22/2020 17:00  NaN    NaN         Fujian        NaN  
4  1/22/2020 17:00  NaN    NaN          Gansu        NaN  
Index(['Active', 'Admin2', 'Combined_Key', 'Confirmed', 'Country_Region',
       'Deaths', 'FIPS', 'Last_Update', 'Lat', 'Long_', 'Province_State',
       'Recovered'],
      dt

In [12]:
## Make update date column
combine_df['Last_Update_date'] = [str(d).split(' ')[0] for d in combine_df['Last_Update']]
combine_df.head()


,Active,Admin2,Combined_Key,Confirmed,Country_Region,Deaths,FIPS,Last_Update,Lat,Long_,Province_State,Recovered,Last_Update_date
0,NaN,NaN,NaN,1.0,Mainland China,NaN,NaN,1/22/2020 17:00,NaN,NaN,Anhui,NaN,1/22/2020
1,NaN,NaN,NaN,14.0,Mainland China,NaN,NaN,1/22/2020 17:00,NaN,NaN,Beijing,NaN,1/22/2020
2,NaN,NaN,NaN,6.0,Mainland China,NaN,NaN,1/22/2020 17:00,NaN,NaN,Chongqing,NaN,1/22/2020
3,NaN,NaN,NaN,1.0,Mainland China,NaN,NaN,1/22/2020 17:00,NaN,NaN,Fujian,NaN,1/22/2020
4,NaN,NaN,NaN,NaN,Mainland China,NaN,NaN,1/22/2020 17:00,NaN,NaN,Gansu,NaN,1/22/2020


In [13]:
select_col = ['Country_Region',  'Province_State','Admin2','Last_Update', 'Confirmed', 'Deaths', 'Recovered', 'Last_Update_date']
combine_subset = combine_df[select_col]

In [14]:
country = 'US'
state = ['California','CA']
state = '|'.join(str(x) for x in state)
print(f'Joint state: {state}')

print(combine_subset[(combine_subset['Country_Region']=='US')]['Province_State'].unique())
print(combine_subset[(combine_subset['Province_State'].str.contains(state, na=False))].shape)
print(combine_subset[(combine_subset['Province_State'].str.contains(state, na=False))])




Joint state: California|CA
['Washington' 'Chicago' 'Illinois' 'California' 'Arizona' 'Chicago, IL'
 'Boston, MA' 'Los Angeles, CA' 'Orange, CA' 'Santa Clara, CA'
 'Seattle, WA' 'Tempe, AZ' 'San Benito, CA' 'Madison, WI'
 'San Diego County, CA' 'San Antonio, TX' 'Ashland, NE' 'Travis, CA'
 'Lackland, TX' 'Humboldt County, CA' 'Sacramento County, CA'
 'Omaha, NE (From Diamond Princess)' 'Travis, CA (From Diamond Princess)'
 'Lackland, TX (From Diamond Princess)'
 'Unassigned Location (From Diamond Princess)' 'Portland, OR'
 'Snohomish County, WA' 'Providence, RI' 'King County, WA'
 'Cook County, IL' 'Grafton County, NH' 'Hillsborough, FL'
 'New York City, NY' 'Placer County, CA' 'San Mateo, CA' 'Sarasota, FL'
 'Sonoma County, CA' 'Umatilla, OR' 'Fulton County, GA'
 'Washington County, OR' ' Norfolk County, MA' 'Berkeley, CA'
 'Maricopa County, AZ' 'Wake County, NC' 'Westchester County, NY'
 'Orange County, CA' 'Contra Costa County, CA' 'Bergen County, NJ'
 'Harris County, TX' 'San Franci

In [16]:
combine_CA = combine_subset[(combine_subset['Province_State'].str.contains(state, na=False))]

combine_CA.groupby(['Last_Update_date']).agg([np.sum])

,Country_Region,Province_State,Admin2,Last_Update,Confirmed,Deaths,Recovered
,sum,sum,sum,sum,sum,sum,sum
Last_Update_date,,,,,,,
1/26/20,US,California,0,1/26/20 16:00,2.0,0.0,0.0
1/27/20,US,California,0,1/27/20 23:59,2.0,0.0,0.0
1/28/20,US,California,0,1/28/20 23:00,2.0,0.0,0.0
1/29/20,US,California,0,1/29/20 19:30,2.0,0.0,0.0
1/30/20,US,California,0,1/30/20 16:00,2.0,0.0,0.0
1/31/2020,US,California,0,1/31/2020 23:59,2.0,0.0,0.0
2/1/2020,USUSUS,"Los Angeles, CAOrange, CASanta Clara, CA",0,2/1/2020 19:532/1/2020 19:532/1/2020 19:53,3.0,0.0,0.0
2020-02-01T19:53:03,USUSUSUSUSUSUSUSUSUSUSUSUSUSUSUSUSUSUSUSUSUSUS...,"Los Angeles, CAOrange, CASanta Clara, CALos An...",0,2020-02-01T19:53:032020-02-01T19:53:032020-02-...,63.0,0.0,0.0
